<a href="https://colab.research.google.com/github/vbejaranos/Data-Science_Habi/blob/main/Habi_Concurso.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **EL ALGORITMO ES CORRECTO**

*Objetivo*:

Teniendo como insumo las bases de datos entregadas por la empresa Habi, se quiere construir, modelar y predecir el precio de oferta (asking price) que Habi debe dar al inmueble que desea adquirir.

***Integrantes***

*Estadísticas Universidad Nacional de Colombia*
- Sara Lucia Acosta, slacostap@unal.edu.co
- Valeria Bejarano Salcedo, vbejaranos@unal.edu.co
- Laura Camila Cruz, laccruzpa@unal.edu.co

## Modulos y librerias




In [ ]:
#!pip install geopandas
#!pip install pyshp

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr"
# Trabajo con tablas y arreglos ==================================================================================#
import pandas as pd
import numpy as np
from math import sqrt
# Librerias para graficos ========================================================================================#
import matplotlib as plt
from matplotlib.pylab import rcParams
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from scipy import stats
def _color_red_or_green(val):
    color = 'red' if np.abs(val) < 0.4 else 'green'
    return 'color: %s' % color
# Librerias para georeferencia ===================================================================================#
#https://geopandas.org/docs/user_guide/set_operations.html
#from shapely.geometry import Polygon
#import shapefile
#import fiona
# Librerias para el modelamiento =================================================================================#
import sklearn as skl
from sklearn.linear_model import LinearRegression # Regresión lineal
from sklearn.svm import SVR # Support Vector Regression
from sklearn.neural_network import MLPRegressor # Redes Neuronales
from sklearn.datasets import make_regression # Redes Neuronales
from sklearn.impute import KNNImputer # Imputacion knn
# Librerías para validación de modelos ===========================================================================#
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
def starMV(x): return 0.0001 if x<1 else x
from sklearn.model_selection import cross_val_score
#Guardar resultados ===============================================================================================# 
import joblib
import os
#from google.colab import files
#uploaded = files.upload()

## Conexión Google Drive con Colab 

Para una mejor gestión de los datos. Al correr el código es necesario entrar al link, dar los permisos que solicitan, copiar el código, pegar en la celda vacía y dar enter. Así solo es necesario compartir una carpeta de datos (shp)

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


# Completar la base de datos

Sabemos que uno de los mejores insumos para el modelamiento son los datos de buena calidad, luego bajo la filosofia de *garbage in, garbage out* se realiza una recopilación y procesamiento de la información de otras [fuentes](https://mapas.bogota.gov.co/) que nos brindan otro tipo de información necesaria dentro de este negocio inmobiliario.

## Datos de catastro

Se obtuvieron las bases de datos correspondientes a [estrato](https://mapas.bogota.gov.co/?l=22935&e=-74.12749288197958,4.696505266929589,-73.96269796010482,4.789654789240459,4686&b=7256#) y [avaluo](https://mapas.bogota.gov.co/?l=10462&e=-74.15372037610308,4.633364461430024,-74.0713229151657,4.674812311999409,4686&b=7256#) de la base catastral de Bogotá, con ello se cruzan por medio de la libreria geopandas con la latitud y longitud de cada inmueble dadas por Habi.

In [ ]:
#import geopandas
#data = pd.read_csv('/content/gdrive/MyDrive/shp/train_data.csv')
#points = data[['id','latitud', 'longitud']]
#catastro = geopandas.read_file('/content/gdrive/MyDrive/shp/Valor_Ref_M.shp')
#catastroM = geopandas.read_file('/content/gdrive/MyDrive/shp/ManzanaEstratificacion.shp')

In [ ]:
#catastro[['id2']]= catastro.index
#catastroM[['id3']]= catastroM.index
#gdf = geopandas.GeoDataFrame({ 'geometry':geopandas.points_from_xy(points.longitud, points.latitud), 
#                             'id': points.id})
#catastro['ANO2'] = [i[0:4] for i in catastro.ANO]
#catastro2 = catastro[catastro.ANO2 == '2020']
#catastroM['FECHA_CAPT2'] = [i[0:4] for i in catastroM.FECHA_CAPT]
#catastroM2 = catastroM[catastroM.FECHA_CAPT2 == '2017']
#poly = geopandas.GeoDataFrame({'geometry':catastro2['geometry'],
#                              'id2': catastro2.id2 })
#poly2 = geopandas.GeoDataFrame({'geometry':catastroM2['geometry'],
#                              'id3': catastroM2.id3 })

In [ ]:
#res_intersection = geopandas.overlay(gdf, poly, how='intersection')
#res_intersection2 = geopandas.overlay(res_intersection, poly2, how='intersection')
#res_intersection3 = res_intersection2[['id', 'id2', 'id3']]
#join = res_intersection3.merge(catastro2, on='id2', how='left')
#join = join.merge(catastroM2, on='id3', how='left')
#exportar = data.merge(join, on ='id', how= 'left')
#exportar.to_csv('/content/gdrive/MyDrive/shp/train_dataV02.csv')

### Bases de datos completada por valor catastral
Una vez se realiza la intersección con los multipoligonos dados por catastro se obtienen las bases V02, a continuación se realizan la imputación de valores faltantes.

In [ ]:
data_train = pd.read_csv('/content/gdrive/MyDrive/shp/train_dataV02.csv')
data_test = pd.read_csv('/content/gdrive/MyDrive/shp/test_dataV02.csv')
data_train[['tipo_data']]= "train"
data_test[['tipo_data']] = "test"
data_train = data_train.drop(['Unnamed: 0'],axis = 1)
data_test = data_test.drop(['Unnamed: 0'],axis = 1)
consol = data_train.append(data_test)

In [ ]:
consol.columns

Index(['id', 'area', 'ascensor', 'balcon', 'banos', 'banoservicio',
       'conjuntocerrado', 'cuartoservicio', 'deposito', 'estrato', 'estudio',
       'garajes', 'gimnasio', 'habitaciones', 'halldealcobas',
       'parqueaderovisitantes', 'piscina', 'piso', 'porteria', 'remodelado',
       'saloncomunal', 'terraza', 'tiempodeconstruido', 'vigilancia', 'vista',
       'zonalavanderia', 'valoradministracion', 'valorventa', 'tipoinmueble',
       'tiponegocio', 'latitud', 'longitud', 'id2', 'id3', 'MANCODIGO',
       'V_REF', 'ANO', 'SHAPE_AREA', 'SHAPE_LEN', 'geometry_x', 'ANO2',
       'CODIGO_MAN', 'ESTRATO', 'CODIGO_ZON', 'CODIGO_CRI', 'NORMATIVA',
       'ACTO_ADMIN', 'NUMERO_ACT', 'FECHA_ACTO', 'ESCALA_CAP', 'FECHA_CAPT',
       'RESPONSABL', 'SHAPE_Leng', 'SHAPE_Area', 'geometry_y', 'FECHA_CAPT2',
       'tipo_data'],
      dtype='object')

## Imputación de datos



Se completan los valores faltantes mediante los siguientes criterios:
1. Las siguientes variables que tienen valores faltantes se completan con 0, pues usualmente son los que el propietario tiene más en cuenta y si no se encuentran en la base, lo más seguro es que no se encuentren en el inmueble.
- Baño de servicio
- Ascensor
- Balcon
- Cuarto de servicio
- Gimnasio
- Piscina 
- Terraza 
- Remodelado
2. Para las siguientes variables se usa imputación de datos via knn (k-nearest-neighbors) para k = 5, mediante las variables de latitud y longitud, es decir, solo por localización del inmueble, pues es bastante usual que las construcciones aledañas contengan la misma arquitectura como decía Juan Sebastian en la conferencia *Uso de la ciencia de datos en la resolución de problemas del sector real*.
- Area
- Baños 
- Estrato 
- Garajes 
- Habitaciones 
- Piso 
- Salon comunal 
- Vigilancia 
- Parqueadero visitantes
- Valor administracion
- V_REF
3. Otras variables dummys, se condicionaron sobre el tipo de inmueble o conjunto para completar 0 o 1 o bien utilizar knn. Esto ya que variables como el pertenecer a un conjunto cerrado o ser un apartamento limita la aparición de estas variables.
- Deposito: apto = 0, casa = 1
- Estudio: apto = 0, casa = 1
- Porteria: conjunto cerrado = 1, no cerrado = 0
- Zona lavanderia: casas = 1,apto = knn
- Hall de alcobas: habitaciones>1 = knn,habitaciones<=1 = 0

### Completar faltantes

In [ ]:
consol[['banoservicio', 'ascensor', 'balcon', 'cuartoservicio', 'gimnasio', 'piscina', 'terraza', 'remodelado']] = consol[['banoservicio', 'ascensor', 'balcon', 'cuartoservicio', 'gimnasio', 'piscina', 'terraza', 'remodelado']].fillna(0)

In [ ]:
consol['tiponegocio'] = consol['tiponegocio'].str.lower()

In [ ]:
consol['tiempodeconstruido'] = consol['tiempodeconstruido'].replace('ntre 0 y 5 años', 'Entre 0 y 5 años')
consol['tiempodeconstruido'].unique()

array([nan, 'Entre 10 y 20 años', 'Entre 0 y 5 años', 'Entre 5 y 10 años',
       'Más de 20 años', '1 a 8 años', '16 a 30 años', '9 a 15 años',
       'Más de 30 años', 'Menos de 1 año', 'Remodelado'], dtype=object)

In [ ]:
apto = consol[consol.tipoinmueble=='Apartamento']
casa = consol[consol.tipoinmueble=='Casa']

In [ ]:
apto[['deposito', 'estudio']] = apto[['deposito', 'estudio']].fillna(0)
casa[['deposito', 'estudio']]= casa[['deposito', 'estudio']].fillna(1)
casa[['zonalavanderia']]= casa[['zonalavanderia']].fillna(1)

In [ ]:
consol = apto.append(casa)

In [ ]:
cerrado = consol[consol.conjuntocerrado==1]
no_cerrado = consol[consol.conjuntocerrado==0]

In [ ]:
cerrado['porteria'] = cerrado['porteria'].fillna(1)
no_cerrado['porteria'] = no_cerrado['porteria'].fillna(0)
cerrado['saloncomunal'] = cerrado['saloncomunal'].fillna(1)

In [ ]:
consol= cerrado.append(no_cerrado)

In [ ]:
consol.loc[consol['valoradministracion']<10000, 'valoradministracion'] = float("NaN")

In [ ]:
train = consol[consol.tipo_data=="train"]
test = consol[consol.tipo_data=="test"]

In [ ]:
train.to_csv('/content/gdrive/MyDrive/shp/train_dataV03.csv')
test.to_csv('/content/gdrive/MyDrive/shp/test_dataV03.csv')

### Imputación via knn

Podemos ver la implementación de la metodología [aquí](https://medium.com/@kyawsawhtoon/a-guide-to-knn-imputation-95e2dc496e)

In [ ]:
data_train = pd.read_csv('/content/gdrive/MyDrive/shp/train_dataV03.csv')
data_test = pd.read_csv('/content/gdrive/MyDrive/shp/test_dataV03.csv')
data_train = data_train.drop(['Unnamed: 0'],axis = 1)
data_test = data_test.drop(['Unnamed: 0'],axis = 1)

In [ ]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors = 5)

In [ ]:
consol1 = data_train.append(data_test)

In [ ]:
area = consol1.loc[:,['area','longitud','latitud']].values
df_area = pd.DataFrame(imputer.fit_transform(area),columns = [[ 'area','longitud','latitud']])

In [ ]:
banos = consol1.loc[:,['banos','longitud','latitud']].values
df_banos = pd.DataFrame(imputer.fit_transform(banos),columns = [[ 'banos','longitud','latitud']])
df_banos.loc[:, ['banos']] = np.round(df_banos.loc[:, ['banos']] )

In [ ]:
estrato = consol1.loc[:,['estrato','longitud','latitud']].values
df_estrato = pd.DataFrame(imputer.fit_transform(estrato),columns = [[ 'estrato','longitud','latitud']])
df_estrato.loc[:, ['estrato']] = np.round(df_estrato.loc[:, ['estrato']] )

In [ ]:
garajes = consol1.loc[:,['garajes','longitud','latitud']].values
df_garajes = pd.DataFrame(imputer.fit_transform(garajes),columns = [[ 'garajes','longitud','latitud']])
df_garajes.loc[:, ['garajes']] = np.round(df_garajes.loc[:, ['garajes']] )

In [ ]:
piso = consol1.loc[:,['piso','longitud','latitud']].values
df_piso = pd.DataFrame(imputer.fit_transform(piso),columns = [[ 'piso','longitud','latitud']])
df_piso.loc[:, ['piso']] = np.round(df_piso.loc[:, ['piso']] )

In [ ]:
habitaciones = consol1.loc[:,['habitaciones','longitud','latitud']].values
df_habitaciones = pd.DataFrame(imputer.fit_transform(habitaciones),columns = [[ 'habitaciones','longitud','latitud']])
df_habitaciones.loc[:, ['habitaciones']] = np.round(df_habitaciones.loc[:, ['habitaciones']] )

In [ ]:
saloncomunal = consol1.loc[:,['saloncomunal','longitud','latitud']].values
df_saloncomunal = pd.DataFrame(imputer.fit_transform(saloncomunal),columns = [[ 'saloncomunal','longitud','latitud']])
df_saloncomunal.loc[:, ['saloncomunal']] = np.round(df_saloncomunal.loc[:, ['saloncomunal']] )

In [ ]:
vigilancia = consol1.loc[:,['vigilancia','longitud','latitud']].values
df_vigilancia = pd.DataFrame(imputer.fit_transform(vigilancia),columns = [[ 'vigilancia','longitud','latitud']])
df_vigilancia.loc[:, ['vigilancia']] = np.round(df_vigilancia.loc[:, ['vigilancia']] )

In [ ]:
zonalavanderia = consol1.loc[:,['zonalavanderia','longitud','latitud']].values
df_zonalavanderia = pd.DataFrame(imputer.fit_transform(zonalavanderia),columns = [[ 'zonalavanderia','longitud','latitud']])
df_zonalavanderia.loc[:, ['zonalavanderia']] = np.round(df_zonalavanderia.loc[:, ['zonalavanderia']] )

In [ ]:
habi_menor_1 = consol1.loc[consol1.loc[ : , 'habitaciones'] <= 1]
habi_mayor_1 = consol1.loc[consol1.loc[ : , 'habitaciones']  > 1]

In [ ]:
habi_menor_1[['halldealcobas']] = habi_menor_1[['halldealcobas']].fillna(0)
consol1= habi_menor_1.append(habi_mayor_1)

In [ ]:
halldealcobas = consol1.loc[: , ['halldealcobas','longitud','latitud']].values
df_halldealcobas = pd.DataFrame(imputer.fit_transform(halldealcobas),columns = [[ 'halldealcobas','longitud','latitud']])
df_halldealcobas.loc[:, ['halldealcobas']] = np.round(df_halldealcobas.loc[:, ['halldealcobas']] )

In [ ]:
parqueaderovisitantes = consol1.loc[: , ['parqueaderovisitantes','longitud','latitud']].values
df_parqueaderovisitantes = pd.DataFrame(imputer.fit_transform(parqueaderovisitantes),columns = [[ 'parqueaderovisitantes','longitud','latitud']])
df_parqueaderovisitantes.loc[:, ['parqueaderovisitantes']] = np.round(df_parqueaderovisitantes.loc[:, ['parqueaderovisitantes']] )

In [ ]:
valoradministracion = consol1.loc[: , ['valoradministracion','longitud','latitud']].values
df_valoradministracion = pd.DataFrame(imputer.fit_transform(valoradministracion),columns = [[ 'valoradministracion','longitud','latitud']])
df_valoradministracion.loc[:, ['valoradministracion']] = np.round(df_valoradministracion.loc[:, ['valoradministracion']] )

In [ ]:
valorref = consol1.loc[: , ['V_REF','longitud','latitud']].values
df_valorref = pd.DataFrame(imputer.fit_transform(valorref),columns = [[ 'V_REF','longitud','latitud']])
df_valorref.loc[:, ['V_REF']] = np.round(df_valorref.loc[:, ['V_REF']] )

In [ ]:
consol1['tiempodeconstruido'].unique()

array(['Entre 0 y 5 años', 'Entre 10 y 20 años', '1 a 8 años',
       'Menos de 1 año', '16 a 30 años', '9 a 15 años',
       'Entre 5 y 10 años', nan, 'Más de 30 años', 'Más de 20 años',
       'Remodelado'], dtype=object)

Realizamos la siguiente codificación:
* Menos de 1 año -- 1
* Entre 0 y 5 años como --  2
* Enre 1 - 8 años grupo -- 3
* Entre 5 y 10 años -- 4
* 9 a 15 años  -- 5
* Entre 10 y 20 años -- 6
* 16 a 30 años' -- 7
* Más de 20 años -- 8
* Más de 30 años  -- 9
*Remodelado -- 10

In [ ]:
consol1['tiempodeconstruido'] = consol1['tiempodeconstruido'].replace( 'Menos de 1 año',1)
consol1['tiempodeconstruido'] = consol1['tiempodeconstruido'].replace( 'Entre 0 y 5 años',2)
consol1['tiempodeconstruido'] = consol1['tiempodeconstruido'].replace( '1 a 8 años',3)
consol1['tiempodeconstruido'] = consol1['tiempodeconstruido'].replace( 'Entre 5 y 10 años',4)
consol1['tiempodeconstruido'] = consol1['tiempodeconstruido'].replace( '9 a 15 años',5)
consol1['tiempodeconstruido'] = consol1['tiempodeconstruido'].replace( 'Entre 10 y 20 años',6)
consol1['tiempodeconstruido'] = consol1['tiempodeconstruido'].replace( '16 a 30 años',7)
consol1['tiempodeconstruido'] = consol1['tiempodeconstruido'].replace( 'Más de 20 años',8)
consol1['tiempodeconstruido'] = consol1['tiempodeconstruido'].replace( 'Más de 30 años',9)
consol1['tiempodeconstruido'] = consol1['tiempodeconstruido'].replace( 'Remodelado',10)

consol1['tiempodeconstruido'].unique()

array([ 2.,  6.,  3.,  1.,  7.,  5.,  4., nan,  9.,  8., 10.])

Se cambio 
* Exterior -- 1
* Interior -- 2

In [ ]:
consol1['vista'] = consol1['vista'].replace( 'Exterior',1)
consol1['vista'] = consol1['vista'].replace( 'Interior',2)

In [ ]:
tiempodeconstruido = consol1.loc[:,['tiempodeconstruido','longitud','latitud']].values
df_tiempodeconstruido = pd.DataFrame(imputer.fit_transform(tiempodeconstruido),columns = [[ 'tiempodeconstruido','longitud','latitud']])
df_tiempodeconstruido.loc[:, ['tiempodeconstruido']] = np.round(df_tiempodeconstruido.loc[:, ['tiempodeconstruido']] )

In [ ]:
vista = consol1.loc[:,['vista','longitud','latitud']].values
df_vista = pd.DataFrame(imputer.fit_transform(vista),columns = [[ 'vista','longitud','latitud']])
df_vista.loc[:, ['vista']] = np.round(df_vista.loc[:, ['vista']] )

In [ ]:
consol1[['area']]       = df_area[['area']]
consol1[['banos']]    = df_banos[['banos']]
consol1[['estrato']]   = df_estrato[['estrato']]
consol1[['garajes']]   = df_garajes[['garajes']]
consol1[['habitaciones']] = df_habitaciones[['habitaciones']]
consol1[['piso']] = df_piso[['piso']]
consol1[['saloncomunal']] = df_saloncomunal[['saloncomunal']]
consol1[['vigilancia']] = df_vigilancia[['vigilancia']]
consol1[['zonalavanderia']] = df_zonalavanderia[['zonalavanderia']]
consol1[['halldealcobas']] = df_halldealcobas[['halldealcobas']]
consol1[['parqueaderovisitantes']] = df_parqueaderovisitantes[['parqueaderovisitantes']]
consol1[['valoradministracion']] = df_valoradministracion[['valoradministracion']]
consol1[['V_REF']] = df_valorref[['V_REF']]
consol1[['tiempodeconstruido']] = df_tiempodeconstruido[['tiempodeconstruido']]
consol1[['vista']] = df_vista[['vista']]

In [ ]:
consol1['tiempodeconstruido'] = consol1['tiempodeconstruido'].replace(1, 'Menos de 1 año')
consol1['tiempodeconstruido'] = consol1['tiempodeconstruido'].replace( 2,'Entre 0 y 5 años')
consol1['tiempodeconstruido'] = consol1['tiempodeconstruido'].replace( 3,'1 a 8 años')
consol1['tiempodeconstruido'] = consol1['tiempodeconstruido'].replace(4, 'Entre 5 y 10 años')
consol1['tiempodeconstruido'] = consol1['tiempodeconstruido'].replace(5, '9 a 15 años')
consol1['tiempodeconstruido'] = consol1['tiempodeconstruido'].replace(6, 'Entre 10 y 20 años')
consol1['tiempodeconstruido'] = consol1['tiempodeconstruido'].replace( 7,'16 a 30 años')
consol1['tiempodeconstruido'] = consol1['tiempodeconstruido'].replace(8, 'Más de 20 años')
consol1['tiempodeconstruido'] = consol1['tiempodeconstruido'].replace( 9,'Más de 30 años')
consol1['tiempodeconstruido'] = consol1['tiempodeconstruido'].replace(10, 'Remodelado')

consol1['tiempodeconstruido'].unique()

array(['1 a 8 años', 'Entre 10 y 20 años', '16 a 30 años', '9 a 15 años',
       'Menos de 1 año', 'Entre 0 y 5 años', 'Entre 5 y 10 años',
       'Más de 20 años', 'Remodelado', 'Más de 30 años', nan],
      dtype=object)

In [ ]:
consol1['vista'] = consol1['vista'].replace(1, 'Exterior')
consol1['vista'] = consol1['vista'].replace( 2,'Interior')

id                         0
area                       0
ascensor                   0
balcon                     0
banos                      0
banoservicio               0
conjuntocerrado            0
cuartoservicio             0
deposito                   0
estrato                    0
estudio                    0
garajes                    0
gimnasio                   0
habitaciones               0
halldealcobas            704
parqueaderovisitantes    704
piscina                    0
piso                       0
porteria                   0
remodelado                 0
saloncomunal               0
terraza                    0
tiempodeconstruido       704
vigilancia                 0
vista                    704
zonalavanderia             0
valoradministracion      704
valorventa                 0
tipoinmueble               0
tiponegocio              770
latitud                    0
longitud                   0
V_REF                    704
tipo_data                  0
dtype: int64

In [ ]:
train1 = consol1[consol1.tipo_data=="train"]
test1 = consol1[consol1.tipo_data=="test"]

In [ ]:
train1.to_csv('/content/gdrive/MyDrive/shp/train_dataV04.csv')
test1.to_csv('/content/gdrive/MyDrive/shp/test_dataV04.csv')

## Datos finales 

In [ ]:
data_train = pd.read_csv('/content/gdrive/MyDrive/shp/train_dataV04.csv')
data_test = pd.read_csv('/content/gdrive/MyDrive/shp/test_dataV04.csv')
data_train = data_train.drop(['Unnamed: 0'],axis = 1)
data_test = data_test.drop(['Unnamed: 0'],axis = 1)
data_train[['tipo_data']]= "train"
data_test[['tipo_data']] = "test"
consolF = data_train.append(data_test)

In [ ]:
consolF[['y']] = consolF[['V_REF']] - consolF[['valorventa']]


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,id,area,ascensor,balcon,banos,banoservicio,conjuntocerrado,cuartoservicio,deposito,estrato,estudio,garajes,gimnasio,habitaciones,halldealcobas,parqueaderovisitantes,piscina,piso,porteria,remodelado,saloncomunal,terraza,tiempodeconstruido,vigilancia,vista,zonalavanderia,valoradministracion,valorventa,tipoinmueble,tiponegocio,latitud,longitud,id2,MANCODIGO,V_REF,ANO,SHAPE_AREA,SHAPE_LEN,geometry,Ano2,tipo_data
0,0,0,335,335,363085,795394273.2,0.0,0.0,1.0,0.0,1,0.0,0.0,2.0,0.0,0.6,0.0,2.0,0.0,1.0,0.0,5.0,1.0,0.0,1.0,0.0,Entre 0 y 5 años,1.0,NaN,1.0,55000.0,1.160000e+08,Apartamento,venta,4.651098,-74.168666,380234.0,6518014.0,1850000.0,2020-01-01,1.935570e-06,0.005504,"POLYGON ((-74.168539409 4.650990077000017, -74...",2020.0,train
1,1,1,1387,1387,378687,0.0,1.0,0.0,0.0,0.0,1,1.0,0.0,6.0,1.0,0.0,0.0,4.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,Entre 10 y 20 años,1.0,Exterior,1.0,745200.0,1.590000e+09,Apartamento,venta,4.690135,-74.049683,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train
2,2,2,1392,1392,213992,345.0,0.0,0.0,0.0,0.0,1,0.0,0.0,6.0,1.0,2.0,1.0,3.0,1.0,1.0,0.0,4.0,1.0,1.0,1.0,0.0,Entre 10 y 20 años,1.0,NaN,1.0,1187000.0,1.800000e+09,Apartamento,venta,4.697660,-74.052653,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train
3,3,3,1401,1401,452566,80.0,0.0,0.0,0.0,0.0,1,0.0,0.0,4.0,1.0,1.0,0.0,3.0,0.0,1.0,0.0,4.0,1.0,0.0,1.0,0.0,Entre 0 y 5 años,1.0,NaN,1.0,190000.0,3.400000e+08,Apartamento,venta,4.731135,-74.054373,352340.0,9104021.0,2900000.0,2020-01-01,3.127788e-06,0.013924,POLYGON ((-74.05485601800001 4.731215995000014...,2020.0,train
4,4,4,1477,1477,676220,87.0,0.0,0.0,0.0,0.0,1,0.0,0.0,4.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,9.0,1.0,0.0,1.0,0.0,1 a 8 años,1.0,NaN,1.0,200000.0,4.200000e+08,Apartamento,venta,4.642000,-74.113998,394636.0,6306002.0,2000000.0,2020-01-01,8.635009e-06,0.015905,POLYGON ((-74.11382247699999 4.643146832000014...,2020.0,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48,48,48,48,48,726088,48.0,0.0,0.0,2.0,0.0,0,1.0,1.0,3.0,1.0,1.0,0.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,Más de 20 años,1.0,NaN,1.0,222287100.0,7.990000e+08,Casa,venta,4.737032,-74.038290,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test
49,49,49,49,49,752156,65.0,0.0,0.0,2.0,0.0,0,1.0,1.0,3.0,1.0,1.0,0.0,3.0,0.0,1.0,0.0,12.0,0.0,0.0,1.0,0.0,Más de 20 años,1.0,NaN,1.0,150000.0,7.490000e+08,Casa,venta,4.735844,-74.038440,355194.0,8517015.0,2900000.0,2020-01-01,1.910642e-06,0.005839,POLYGON ((-74.03668588800002 4.735051208000016...,2020.0,test
50,50,50,50,50,1055205,63.0,0.0,0.0,2.0,0.0,0,1.0,1.0,3.0,1.0,1.0,0.0,3.0,0.0,1.0,0.0,2.0,0.0,0.0,1.0,0.0,Más de 20 años,1.0,NaN,1.0,118000.0,7.490000e+08,Casa,venta,4.736058,-74.040634,390616.0,8519050.0,2400000.0,2020-01-01,5.794660e-07,0.003272,POLYGON ((-74.04008913899997 4.735252899999978...,2020.0,test
51,51,51,51,51,1289402,53.0,0.0,0.0,2.0,0.0,0,1.0,1.0,2.0,1.0,1.0,0.0,3.0,0.0,1.0,0.0,12.0,0.0,0.0,1.0,0.0,Más de 20 años,1.0,NaN,1.0,70000.0,6.990000e+08,Casa,venta,4.737620,-74.040190,349138.0,8519048.0,2400000.0,2020-01-01,3.883588e-07,0.002686,POLYGON ((-74.03964318099997 4.737262713999996...,2020.0,test


# Estadísticas descriptivas

## Correlaciones lineales entre las variables

In [ ]:
# Gráfica interactiva de las diferentes variables en la tabla data
df = consolF
fig = go.Figure()
fig.add_trace(go.Scatter(x = df.id,y = df[df.columns[0]],visible = True))
updatemenu = []
buttons = []
for col in df.columns:
    buttons.append(dict(method = 'restyle',label = col,visible = True, 
                        args = [{'y':[df[col]],'x':[df.id],'type':'scatter'}, [0]],))
updatemenu = []
your_menu = dict()
updatemenu.append(your_menu)
updatemenu[0]['buttons'] = buttons
updatemenu[0]['direction'] = 'down'
updatemenu[0]['x'] = 0.7
updatemenu[0]['y'] = 1.15
updatemenu[0]['showactive'] = True
fig.update_layout(showlegend = False,updatemenus = updatemenu)
fig.update_layout(yaxis = dict(showexponent = 'all',exponentformat = 'e'))
fig.update_layout(xaxis_title = '', yaxis_title = '',template = 'plotly_white')
fig.show()

In [ ]:
#Calcular correlación entre variables
df = data_train[data_train.id != data_test.id]
correlation_Pearson = df.corr(method='pearson')
i = np.argsort(correlation_Pearson.iloc[0,:]).values
correlation_Pearson = df.iloc[:,i].corr(method='pearson')
correlation_Kendall = df.iloc[:,i].corr(method='kendall')
correlation_Spearman = df.iloc[:,i].corr(method='spearman')

In [ ]:
fig = go.Figure()
fig.add_trace(go.Heatmap(z=correlation_Pearson.values,
                         x=np.arange(0,correlation_Pearson.shape[0]),
                         y=correlation_Pearson.columns,hoverongaps = False))
updatemenu = []
buttons = []
buttons.append(dict(method = 'restyle',label = "Correlación de pearson",visible = True, 
                        args = [{'z':[correlation_Pearson.values],
                                 'x':[np.arange(0,correlation_Pearson.shape[0])],
                                 'y':[correlation_Pearson.columns],
                                 'type':'heatmap'}, [0]],))
buttons.append(dict(method = 'restyle',label = "Correlación de kendall",visible = True, 
                        args = [{'z':[correlation_Kendall.values],
                                 'x':[np.arange(0,correlation_Kendall.shape[0])],
                                 'y':[correlation_Kendall.columns],
                                 'type':'heatmap'}, [0]],))
buttons.append(dict(method = 'restyle',label = "Correlación de spearman",visible = True, 
                        args = [{'z':[correlation_Spearman.values],
                                 'x':[np.arange(0,correlation_Spearman.shape[0])],
                                 'y':[correlation_Spearman.columns],
                                 'type':'heatmap'}, [0]],))
updatemenu = []
your_menu = dict()
updatemenu.append(your_menu)
updatemenu[0]['buttons'] = buttons
updatemenu[0]['direction'] = 'down'
updatemenu[0]['x'] = 0.7
updatemenu[0]['y'] = 1.15
updatemenu[0]['showactive'] = True
fig.update_xaxes(side="top")
fig.update_layout(
    showlegend = False,
    height=500,width = 1000,
    updatemenus = updatemenu,
    xaxis = dict(
        tickmode = 'array',
        tickvals = np.arange(0,correlation_Pearson.shape[0]),
        ticktext = np.arange(0,correlation_Pearson.shape[0])[::-1]
    ),
    legend_orientation="h"
)
fig.show()

In [ ]:
correlations_Tab = pd.DataFrame({'Pearson':correlation_Pearson['y'].values[::-1],
                                'Kendall':correlation_Kendall['y'].values[::-1],
                                'Spearman':correlation_Spearman['y'].values[::-1]})
correlations_Tab.index = correlation_Pearson.index[::-1]
# Tabla de resultados ordenados de acuerdo a la correlación de Pearson
## Marcamos de color verde aquellas correlaciones mayores a 0.4
correlations_Tab.style.applymap(_color_red_or_green)

# Modelamiento

## Validación cruzada

El método de validación cruzada es usado para seleccionar el modelo que mejor se ajuste a los datos. Consiste en dividir los datos en $K$ conjuntos de entrenamiento y de prueba: los modelos propuestos serán ajustados a cada subconjunto de entrenamento y se evaluará su calidad de ajuste con el subconjunto de prueba, de acuerdo a una medida de error o función  de perdida se determina qué modelo se desempeña mejor.

Nordhausen, K. (2014). An Introduction to Statistical Learning—with Applications in R by Gareth James, Daniela Witten, Trevor Hastie & Robert Tibshirani.

In [ ]:
Ny = int(data_train.shape[0]) 
Nk = int(data_train.shape[0]/2)

In [ ]:
# Función para validaciones con parámetros: n (cantidad de datos),min_n (tamaños de lotes)
def split(n,min_n = 3):
     cc = min_n
     min = 0
     while cc != n:
       idx = np.arange(0,cc, dtype=int)
       if (not Rolling):idx = np.arange(min,cc, dtype=int)
       if (cc == n):break
       idy = np.arange(cc,cc + 1, dtype=int)
       if (not Rolling):idy = np.arange(cc,n, dtype=int)
       cc = cc + 1
       min = min + 1
       yield idx, idy
for train, test in split(min_n = Nk,n = Ny):
    print("%s %s" % (train, test))

## Calibración hiperparámetros

Los modelos que vamos a considerar son: Regresión lineal con penalización Lasso, regresor del árbol de decisión (Random Forest), máquina de vectores de soporte de regresión (SVR) y una red Neuronal MLP (Multi Layer Perceptron). Cada uno de ellos tiene hiperparámetros que deben ser encontrados, se usará el método de validación cruzada con ese fin.

Para encontrar los hiperparametros óptimos de un modelo, se compara el modelo con diferentes valores de hiperparametros, es decir, se ajustan distintos modelos de la misma clase que serán comparados entre sí, teniendo como criterio la medida de perdida:  error absoluto medio porcentual (MAPE).

El calculo funciona de la siguiente manera: se define una grilla de valores que podrian tomar los hiperparametros  y una particion de los datos en conjuntos entrenamiento-validación (definidos anteriormente en  kfold) que por ejemplo podria ser: $C_{train} = [3\ 4\ 5\ 6\ 7\ 8\ 9]$ y $C_{test} = [10\ 11\ 12\ 13]$ .

A continuación, con esta grilla de valores se ajusta el modelo para cada hiperparametro propuesto (modelos a comparar) con los datos en  $C_{train}$ para luego encontrar las predicciones de la variable respuesta $\hat{Y}_{C_{test}}$ sobre el conjunto test y compararlas con las reales $Y_{C_{test}}$ mediante la medida de error. 

$$\mathrm{MAPE}=\frac{1}{|C_{test}|} \sum_{i \in C_{test}}\left|\frac{\hat{Y}_{i}-Y_{i}}{Y_{i}}\right|$$

Esta busqueda se realiza mediante la funcion **GridSearchCV** que implementa un método de "ajuste" optimizando mediante la búsqueda de cuadrícula validada cruzada sobre una cuadrícula de parámetros dada.

Para dar un poco de precisión al procedimiento y optimizarlo se realiza una búsqueda en dos faces. En la primera fase buscamos en el intervalo $[a,b]$ con saltos de mayor tamaño donde encontramos un valor optimo $j_1$, luego en la segunda fase se refina la busqueda al rededor del valor encontrado: en $[a,J_1 + mv*j_1]$.

In [ ]:
score = 

### Regresión Lasso

Se intenta penalizar la magnitud en valor absoluto de los **p** coeficientes del vector de parámetros, esta penalización se hace sobre la función de costo.
$$Lasso(\pmb{\beta})=\underbrace{\frac{1}{n}\sum_{i=1}^n(Y_i-\pmb{X}_i\pmb{\beta})^2}_{MSE(\beta)}+\color{red}{\alpha||\pmb{\beta}_i||_1}$$
Lo que equivale a minimizar:
$$Lasso(\pmb{\beta})=\underbrace{\frac{1}{n}\sum_{i=1}^n(Y_i-\pmb{X}_i\pmb{\beta})^2}_{MSE(\beta)}+\alpha\sum_{i=1}^p|\pmb{\beta}_i|$$ 

$\lambda$: coeficiente de penalidad. 

Si $\lambda$ es lo suficiente elevado, algunos de los coeficientes pueden llegar a reducirse hasta exactamente 0. Si $\lambda=0$ se obtiene la solución de minimos cuadrados.


*La estimación de $\hat{\pmb{\beta}}_{Lasso}$ no tiene una forma cerrada, para su estimación se requieren métodos númericos*

In [ ]:
model=skl.linear_model.Lasso(max_iter = 10000)
# Creamos una lista con los parámetros y la grilla de valores a probar
params = {'alpha': np.arange(start=0.0001, stop=1, step=0.01)}
# coeficiente de penalidad (Lambda)
kfold = split(min_n = Nk,n = Ny)
grid = GridSearchCV(model, params, cv = kfold, scoring=score)
grid.fit(data_base['X']['rescaled'], data_base['Y']['rescaled'][:,0])
par_1 = grid.best_params_['alpha']
params['alpha'] = np.arange(start = starMV(par_1 - 0.5), stop = par_1 + 0.5, step = 0.001)
kfold = split(min_n = Nk,n = Ny)
grid = GridSearchCV(model, params, cv=kfold, scoring=score)
grid.fit(data_base['X']['rescaled'], data_base['Y']['rescaled'][:,0])
grids['Lasso' + score] = grid.best_params_
print (score, grid.best_params_)

### Random Forest

Se puede ver en la [guia del usuario](https://scikit-learn.org/stable/modules/tree.html#tree) lo necesario para este modelo, a grandes rasgos se tiene que:

Dados los vectores de entrenamiento $x_i \in \mathbb{R}^p, i=1,...,n$ y un vector $y \in \mathbb{R}^n$, un árbol de decisión divide el espacio de forma recursiva de manera que las muestras con valor similar en $y_i$ se agrupen.
Los datos en el nodo $m$ representados por $Q$, para cada candidato $\theta=(j,t_m)$ que consiste en una caracteristica $j$ y en un umbral $t_m$ que particiona los datos así:

$$Q_{left}(\theta) = {(x, y) | x_j <= t_m}\\Q_{right}(\theta) = Q \setminus Q_{left}(\theta)$$

La impureza en $m$ se calcula utilizando una función de impureza $H()$

$$G(Q, \theta) = \frac{n_{left}}{N_m} H(Q_{left}(\theta))
+ \frac{n_{right}}{N_m} H(Q_{right}(\theta))$$

Que pueden ser 

Error absoluto medio:
$$median(y)_m = \underset{i \in N_m}{\mathrm{median}}(y_i)$$
$$H(X_m) = \frac{1}{N_m} \sum_{i \in N_m} |y_i - median(y)_m|$$

Seleccione los parámetros que minimizan la impureza.

$$\theta^* = \operatorname{argmin}_\theta  G(Q, \theta)$$

Itere para subconjuntos $Q_{left}(\theta^*)$ y $Q_{right}(\theta^*)$ hasta alcanzar la profundidad maxima permitida ($\texttt{max}\_\texttt{depht}$).

In [ ]:
from sklearn.tree import DecisionTreeRegressor
params = {'max_depth': np.round(np.arange(start=1, stop=5, step=1),0)} # La profundidad máxima del árbol
model_DT = DecisionTreeRegressor(splitter= 'best',min_samples_leaf=2,random_state=7)
for score in scoring:
  kfold = split(min_n = Nk,n = Ny)
  grid = GridSearchCV(model_DT, params, cv=kfold,  scoring=score)
  grid.fit(data_base['X']['rescaled'], data_base['Y']['rescaled'][:,0])
  grids['DecisionTree' + score] = grid.best_params_
  print(score, grid.best_params_)

### Support Vector Regression (SVR)

Podemos ir a la [guía de usuario](https://scikit-learn.org/stable/modules/svm.html#svm-regression) para detallar lo necesario del modelo, a grandes rasgos se tiene que:

El problema dual es
$$\min_{\alpha, \alpha^*} \frac{1}{2} (\alpha - \alpha^*)^T Q (\alpha - \alpha^*) + \varepsilon \mathbf{1}^T (\alpha + \alpha^*) - y^T (\alpha - \alpha^*)$$
sujeto a $$\mathbf{1}^T (\alpha - \alpha^*) = 0$$
$$ 0 \leq \alpha_i, \alpha_i^* \leq C, i=1, ..., n$$
 
donde $\mathbf{1}$ es el vector de unos, $Q$ es una matriz semidefinida positiva $n$ por $n$, $Q_{ij}=K(x_i,x_j)=\phi(x_i)^T\phi(x_j)$ es el núcleo. Aquí los vectores de entrenamiento se mapean implícitamente en un espacio dimensional más alto por la función $\phi$.

La predicción es:
$$\sum_{i \in SV}(\alpha_i - \alpha_i^*) K(x_i, x) + b$$

Se puede acceder a estos parámetros a través de los atributos $\texttt{dual}\_\texttt{coef}\_$ que contiene la diferencia $\alpha_i-\alpha_i^*$, $\texttt{support}\_\texttt{vectors}\_$ que contiene los vectores de soporte y $\texttt{intercept}\_$ que contiene el término independiente $b$.

In [ ]:
params_i={}
  iter_max=10000
  param = [{'kernel': ['rbf'], #Los hiperparámetros para este kernel son gamma y C
            'gamma':np.arange(start=0.0001, stop=10, step=2),
            'C':np.arange(start=0.0001, stop=5, step=1),},
          {'kernel': ['poly'], #Los hiperparámetros para este kernerl son gammas, coef0 (intercepto), degree (grados polinomio) y C
            'gamma': np.arange(start=0.0001, stop=10, step=2),
            'coef0': np.arange(start=0.0001, stop=5, step=1), 
            'degree': (2, 3 , 4),
            'C':np.arange(start=0.0001, stop=5, step=1),},
          {'kernel': ['sigmoid'], #Los hiperparámetros para este kernel son coef0 (intercepto), C y gamma
          'coef0':np.arange(start=0.0001, stop=5, step=1),
          'C':np.arange(start=0.0001, stop=5, step=1),
          'gamma':np.arange(start=0.0001, stop=10, step=2)}]
  model_svm = SVR(max_iter = iter_max,epsilon = 0.01)
  for score in scoring:
    kfold = split(min_n = Nk,n = Ny)
    grid_i = GridSearchCV(model_svm, param, cv=kfold, scoring=score)
    grid_i.fit(data_train[vars], data_train['y'][:,0])
    par_1 =  grid_i.best_params_['gamma']
    params_i['gamma'] = np.arange(start=starMV(par_1 - 0.1*10), stop=par_1 + 0.1*10, step=0.1)
    par_2 =  grid_i.best_params_['C']
    params_i['C'] = np.arange(start=starMV(par_2 - 0.1*10), stop=par_2 + 0.1*10, step=0.1)
    model_svm = SVR(max_iter=iter_max,epsilon = 0.01,kernel='rbf')
    kernel_i = grid_i.best_params_['kernel']
    if kernel_i == 'sigmoid':
      par_3 =  grid_i.best_params_['coef0']
      params_i['coef0'] = np.arange(start=starMV(par_3 - 0.1*10), stop=par_3 + 0.1*10, step=0.1)
      model_svm = SVR(max_iter=iter_max,epsilon = 0.01,kernel='sigmoid')
    if kernel_i == 'poly':
      par_3 =  grid_i.best_params_['coef0']
      params_i['coef0'] = np.arange(start=starMV(par_3 - 0.1*10), stop=par_3 + 0.1*10, step=0.1)
      degree_i = grid_i.best_params_['degree']
      model_svm = SVR(max_iter=iter_max,epsilon = 0.01,kernel='poly',degree=degree_i)
    kfold = split(min_n = Nk,n = Ny)
    grid_i = GridSearchCV(model_svm, params_i, cv=kfold, scoring=score)
    grid_i.fit(data_train[vars], data_train['y'][:,0])
    grids['SVM' + score] = grid_i.best_params_
    grids['SVM' + score]['kernel'] = kernel_i
    if kernel_i == 'poly':
      grids['SVM' + score]['degree'] = degree_i
    print (score, grids['SVM' + score])

### Multi Layer Perceptron (MLP)

Las redes neuronales artificiales (ANN) son una clase de
modelos utilizados para tareas de clasificación y regresión no
lineal que están motivadas por el cálculo neuronal biológico.

Cada componente $x_i$ de la entrada se multiplica por un peso correspondiente $w_i$, se añade un sesgo y se aplica una función de activación. Los pesos  y sesgos son la información utilizada por la red neuronal para resolver un problema. Estos parámetros deben aprenderse (ajustarse) en el paso de entrenamiento.

Una red **MLP** constituye uno de los modelos más comunes por su condición como aproximador universal de funciones, estas redes utilizan más de una capa 
oculta de neuronas.

De modo que en cada capa j se hará transformaciones lineales del vector de entrada $\vec{x}=(x_1,...,x_N)^T$, se añade un sesgo  y a partir de la función de activación se obtiene un vector de salida $\vec{y}$:
\begin{equation*}
    \vec{y}=\Phi^{(j)} \left(W^{(j)}\vec{x}+\vec{b}^{(j)}\right)
\end{equation*}
 Este $\vec{y}$ será el vector de entrada a la siguiente capa oculta o el resultado en caso de ser la última capa.

In [ ]:
paramMLP = {'solver':['adam'],
          'activation':['logistic','tanh','relu'],
          'hidden_layer_sizes': [(2),(4),(8),(2,4),(4,3),(4,8),(8,2),(4,3,2)],
          'alpha': [0.01,0.001,0.0001,0.00001],
          'learning_rate_init':[0.001,0.01,0.0001,0.1],
          'beta_1':[0.3,0.6,0.9,0.99,0.999],
          'beta_2':[0.3,0.6,0.9,0.99,0.999]}
  modelMLP= MLPRegressor(batch_size='auto',verbose=False,shuffle=False,max_iter=1000,random_state=7,validation_fraction=0)
  best_params=[]
  for score in scoring:
    kfold = split(min_n = Nk,n = Ny)
    gridMLP = GridSearchCV(modelMLP, paramMLP, cv=kfold, scoring=score,n_jobs=-1)
    gridMLP.fit(data_train[vars], data_base['y'][:,0])
    best= score, gridMLP.best_params_
    best_params.append(best)
    grids['MLP' + score] = gridMLP.best_params_
    print(score, gridMLP.best_params_)

In [ ]:
#Descomentar para guardar nueva grilla
saved = joblib.dump(grids, os.path.join('/content/gdrive/MyDrive/shp/Grids Habi.joblib'))
#Carga los parámetros elegidos para ahorrar tiempo de ejecución
#grids = joblib.load('Grids Habi.joblib')

## Modelos finales

In [ ]:
 #Función para iniciazar todos los modelos a competir
def inicializarModelos(criterio):
  modelos = []
  grids_i = grids
  ## Modelo SVR final del script sin modificar
  if grids_i['SVM' + criterio]['kernel'] == 'poly':
    modelos.append(('SVM', SVR(max_iter=1000,kernel = 'poly',
                               coef0 =  grids_i['SVM' + criterio]['coef0'],
                               gamma = grids_i['SVM' + criterio]['gamma'],
                               C = grids_i['SVM' + criterio]['C'],
                               degree = grids_i['SVM' + criterio]['degree'],
                                 epsilon = 0.001)))
  if grids_i['SVM' + criterio]['kernel'] =='rbf':
    modelos.append(('SVM', SVR(max_iter=1000,kernel = 'rbf',
                               gamma = grids_i['SVM' + criterio]['gamma'],
                               C = grids_i['SVM' + criterio]['C'],
                               epsilon = 0.001)))
  if grids_i['SVM' + criterio]['kernel'] =='sigmoid':
    modelos.append(('SVM', SVR(max_iter = 1000,kernel = 'sigmoid',
                               coef0 = grids_i['SVM' + criterio]['coef0'],
                                 gamma = grids_i['SVM' + criterio]['gamma'],
                                 C = grids_i['SVM' + criterio]['C'],
                                 epsilon = 0.001)))
  modelos.append(('MLP',skl.neural_network.MLPRegressor(hidden_layer_sizes = grids_i['MLP' + criterio]['hidden_layer_sizes'],
                                                          activation = grids_i['MLP' + criterio]['activation'],
                                                          solver = grids_i['MLP' + criterio]['solver'],
                                                          alpha = grids_i['MLP' + criterio]['alpha'],
                                                          beta_1 = grids_i['MLP' + criterio]['beta_1'],
                                                          beta_2 = grids_i['MLP' + criterio]['beta_2'],
                                                          learning_rate_init = grids_i['MLP' + criterio]['learning_rate_init'],
                                                          verbose = False,random_state = 7,
                                                          validation_fraction = 0,shuffle = False,max_iter = 1000)))
  return modelos

In [ ]:
# Iniciamos los modelos
Modelos = {}
for j in scoring:
  Modelos[j] = inicializarModelos(criterio = j)
Resultados = {} # Lista para guardar predicciones y medidas de ajuste
for j in scoring:
  Resultados[j] = {}
  results_cv = []
  names_cv = []
  predictions = {}
  for name, modelo in Modelos[j]:
    # Calculamos las medidas de perdida
    kfold = split(min_n = 7,n = 14)
    cv = cross_val_score(modelo,data_train[vars], data_base['y'][:,0],
                         cv = kfold, scoring = j)
    # Ajustamos cada modelo
    modelo.fit(data_train[vars][:],
              data_train['y'][:,0])
    # Realizamos las predicciones
    predictions[name] = modelo.predict(data_train['ypred'])
    results_cv.append(cv)
    names_cv.append(np.repeat(name,cv.shape[0]))
  Resultados[j]['predictions'] = predictions
  Resultados[j]['cv'] = {}
  Resultados[j]['cv']['value'] = results_cv
  Resultados[j]['cv']['names'] = names_cv

In [ ]:
# Modelos para el promedio
Modelos_promedio = ['Lasso','ElasticNet','SVM','MLP','DecisionTree']

In [ ]:
for j in scoring:
  predictions = Resultados[j]['predictions']
  scalerY = data_base['Y']['scaler']
  dfResModels =  pd.DataFrame({'Lasso':INVT(predictions['Lasso'],scalerY),
                            'ElasticNet':INVT(predictions['ElasticNet'],scalerY),
                            'SVM':INVT(predictions['SVM'],scalerY),
                            'MLP':INVT(predictions['MLP'],scalerY),
                            'DecisionTree':INVT(predictions['DecisionTree'],scalerY)})
  dfResModels['medias'] = np.nan
  dfResModels['medias'][Ny:]=dfResModels[Modelos_promedio][Ny:].mean(axis=1)
  dfResModels['Real']=Y.values[Y.index<= prediction_data[1]]
  dfResModels.index = X.index[Y.index<= prediction_data[1]]
  Resultados[j]['dataframe'] = dfResModels

In [ ]:
# Graficar resultado de la proyección
df = Resultados[scoring[0]]['dataframe']
fig = go.Figure()
for i in df.columns:
  fig.add_scatter(x=df.index, y=df[i],mode='lines',name=i)
fig.update_layout(title_text="Predicciones modelos")
fig.update_layout(yaxis = dict(showexponent = 'all',exponentformat = 'e'))
fig.update_layout(xaxis_title = XLABEL, yaxis_title = YLABEL,template = "plotly_white")
fig.show()

In [ ]:
sc = scoring[0]
df = pd.DataFrame({'values': np.reshape(np.array(Resultados[sc]['cv']['value']),-1),
                   'model': np.reshape(np.array(Resultados[sc]['cv']['names']),-1)})
fig = go.Figure()
fig.add_trace(go.Box(y = df['values'],x = df['model'],boxpoints = 'all',boxmean=True))
fig.update_layout(boxmode='group',template = "plotly_white",title_text="Cv " + sc)
fig.show()

In [ ]:
df.groupby('model').mean().sort_values('values',ascending=False).T

In [ ]:
models=Resultados[scoring[0]]['dataframe'].columns
MSE=[]
for i in models[:-2]:
  aux=mean_squared_error(data_base['Y']['rescaled'],Resultados[scoring[0]]['predictions'][i][:Ny])
  MSE.append(aux)
b=pd.DataFrame({'MSE':MSE},index=models[:-2]) 
b.sort_values('MSE',ascending=True).T